In [1]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))
from datetime import datetime

import src.utils as utils
import src.myenv as myenv
import src.calcEMA as calc_utils

import numpy as np
import pandas as pd

from pycaret.classification import ClassificationExperiment

<built-in method __dir__ of sqlite3.Connection object at 0x7f65f9e4a740>


In [2]:
data = utils.get_data(symbol="BTCUSDT", interval="1h", columns=myenv.all_cols)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 55836 entries, 2017-08-17 04:00:00 to 2024-01-04 23:00:00
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     55836 non-null  datetime64[ns]
 1   open                          55836 non-null  float32       
 2   high                          55836 non-null  float32       
 3   low                           55836 non-null  float32       
 4   close                         55836 non-null  float32       
 5   volume                        55836 non-null  float32       
 6   close_time                    55836 non-null  int64         
 7   quote_asset_volume            55836 non-null  float32       
 8   number_of_trades              55836 non-null  int16         
 9   taker_buy_base_asset_volume   55836 non-null  float32       
 10  taker_buy_quote_asset_volume  55836 non-null  float32      

In [3]:
p_ema = 250
min_rsi = 30
max_rsi = 70

data["close_time"] = pd.to_datetime(data["close_time"], unit="ms")
data["amplitude"] = ((data["high"] - data["low"]) / data["low"]) * 100
data["change_open_close"] = ((data["close"] - data["open"]) / data["open"]) * 100
data = calc_utils.calc_ema_periods(
    data, periods_of_time=[i for i in range(50, 250 + 1, 25)]
)
data = calc_utils.calc_RSI(data)

data['close_max'] = data['close'].rolling(24).max()
data.dropna(inplace=True)
data['result'] = ((data['close_max'] - data['close']) / data['close']) * 100
data['label'] = np.where(data['result'] >= 1.5, 'LONG', 'ESTAVEL')

data.drop(columns=['close_max', 'result'], inplace=True)
data.info()
data.tail(10)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 55587 entries, 2017-08-27 13:00:00 to 2024-01-04 23:00:00
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     55587 non-null  datetime64[ns]
 1   open                          55587 non-null  float32       
 2   high                          55587 non-null  float32       
 3   low                           55587 non-null  float32       
 4   close                         55587 non-null  float32       
 5   volume                        55587 non-null  float32       
 6   close_time                    55587 non-null  datetime64[ns]
 7   quote_asset_volume            55587 non-null  float32       
 8   number_of_trades              55587 non-null  int16         
 9   taker_buy_base_asset_volume   55587 non-null  float32       
 10  taker_buy_quote_asset_volume  55587 non-null  float32      

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,...,ema_175p,ema_175p_diff,ema_200p,ema_200p_diff,ema_225p,ema_225p_diff,ema_250p,ema_250p_diff,rsi,label
ix_open_time,,,,,,,,,,,,,,,,,,,,,
2024-01-04 14:00:00,2024-01-04 14:00:00,43346.671875,43753.148438,43250.000000,43676.070312,2649.806641,2024-01-04 14:59:59.999,115171256.0,-32622,1348.764282,...,43373.371094,0.697892,43331.804688,0.794487,43294.035156,0.882420,43258.339844,0.965665,57.271320,ESTAVEL
2024-01-04 15:00:00,2024-01-04 15:00:00,43676.070312,44329.988281,43608.000000,44051.101562,5051.478516,2024-01-04 15:59:59.999,222248016.0,-2564,2602.486084,...,43381.074219,1.544515,43338.960938,1.643188,43300.734375,1.732920,43264.656250,1.817755,62.870918,ESTAVEL
2024-01-04 16:00:00,2024-01-04 16:00:00,44051.101562,44134.800781,43777.859375,43915.710938,2764.450439,2024-01-04 16:59:59.999,121431080.0,-30756,1303.834717,...,43387.148438,1.218247,43344.699219,1.317374,43306.179688,1.407493,43269.843750,1.492650,59.822948,ESTAVEL
2024-01-04 17:00:00,2024-01-04 17:00:00,43915.699219,44225.750000,43827.691406,43990.011719,2510.891846,2024-01-04 17:59:59.999,110519632.0,14798,1163.496460,...,43394.000000,1.373489,43351.121094,1.473758,43312.230469,1.564873,43275.582031,1.650884,60.942028,ESTAVEL
2024-01-04 18:00:00,2024-01-04 18:00:00,43990.011719,44152.089844,43881.308594,44048.960938,1520.186768,2024-01-04 18:59:59.999,66942304.0,-10071,743.466309,...,43401.441406,1.491931,43358.066406,1.593463,43318.750000,1.685669,43281.742188,1.772615,61.849945,ESTAVEL
2024-01-04 19:00:00,2024-01-04 19:00:00,44048.949219,44437.500000,43910.011719,43956.480469,2603.992676,2024-01-04 19:59:59.999,115098296.0,-32560,1304.690430,...,43407.750000,1.264130,43364.019531,1.366250,43324.394531,1.458961,43287.121094,1.546324,59.512707,ESTAVEL
2024-01-04 20:00:00,2024-01-04 20:00:00,43956.480469,44690.000000,43890.898438,44113.679688,3043.389648,2024-01-04 20:59:59.999,134695792.0,-24063,1636.639038,...,43415.773438,1.607495,43371.480469,1.711261,43331.378906,1.805391,43293.707031,1.893977,62.132214,ESTAVEL
2024-01-04 21:00:00,2024-01-04 21:00:00,44113.679688,44589.039062,44113.671875,44430.859375,2538.904785,2024-01-04 21:59:59.999,112601808.0,22836,1325.106445,...,43427.308594,2.310875,43382.019531,2.417683,43341.109375,2.514357,43302.765625,2.605131,66.799690,ESTAVEL
2024-01-04 22:00:00,2024-01-04 22:00:00,44430.859375,44729.578125,44150.640625,44318.000000,2470.819092,2024-01-04 22:59:59.999,109918192.0,18420,1345.939575,...,43437.429688,2.027215,43391.332031,2.135606,43349.753906,2.233568,43310.855469,2.325386,63.786949,ESTAVEL


In [4]:
train_data = data.iloc[:int(len(data) * 0.7)]
validation_data = data.iloc[train_data.shape[0]:]
train_data.tail()

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,...,ema_175p,ema_175p_diff,ema_200p,ema_200p_diff,ema_225p,ema_225p_diff,ema_250p,ema_250p_diff,rsi,label
ix_open_time,,,,,,,,,,,,,,,,,,,,,
2022-02-08 21:00:00,2022-02-08 21:00:00,44271.949219,44393.359375,43849.898438,44180.359375,2330.742920,2022-02-08 21:59:59.999,102923936.0,-12902,1168.724731,...,41119.105469,7.444846,40854.648438,8.140348,40637.046875,8.719415,40460.789062,9.193025,58.693707,LONG
2022-02-08 22:00:00,2022-02-08 22:00:00,44180.000000,44274.691406,43865.218750,44062.890625,924.053955,2022-02-08 22:59:59.999,40718832.0,-30132,461.614197,...,41152.558594,7.072056,40886.570312,7.768615,40667.363281,8.349514,40489.488281,8.825506,56.625362,LONG
2022-02-08 23:00:00,2022-02-08 23:00:00,44062.890625,44299.980469,44013.558594,44042.988281,992.339355,2022-02-08 23:59:59.999,43791224.0,29124,449.261078,...,41185.406250,6.938336,40917.976562,7.637259,40697.238281,8.221073,40517.804688,8.700332,56.263596,LONG
2022-02-09 00:00:00,2022-02-09 00:00:00,44043.000000,44341.878906,43958.699219,44243.988281,1535.225098,2022-02-09 00:59:59.999,67701304.0,-24310,650.385925,...,41220.160156,7.335799,40951.070312,8.041103,40728.625000,8.631186,40547.496094,9.116450,59.105175,ESTAVEL
2022-02-09 01:00:00,2022-02-09 01:00:00,44243.988281,44247.539062,43800.390625,43861.210938,1576.450317,2022-02-09 01:59:59.999,69413008.0,-22857,698.041199,...,41250.171875,6.329765,40980.027344,7.030702,40756.347656,7.618110,40573.898438,8.102037,52.155685,LONG


In [5]:
validation_data.head()

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,...,ema_175p,ema_175p_diff,ema_200p,ema_200p_diff,ema_225p,ema_225p_diff,ema_250p,ema_250p_diff,rsi,label
ix_open_time,,,,,,,,,,,,,,,,,,,,,
2022-02-09 02:00:00,2022-02-09 02:00:00,43861.218750,43926.828125,43117.921875,43340.218750,2187.325684,2022-02-09 02:59:59.999,94943504.0,-10825,960.914307,...,41273.921875,5.006301,41003.511719,5.698798,40779.210938,6.280180,40595.941406,6.759980,44.488361,LONG
2022-02-09 03:00:00,2022-02-09 03:00:00,43340.230469,43488.570312,43252.761719,43343.281250,1023.349792,2022-02-09 03:59:59.999,44392388.0,-29812,473.999756,...,41297.437500,4.953924,41026.792969,5.646282,40801.902344,6.228580,40617.832031,6.709982,44.539978,LONG
2022-02-09 04:00:00,2022-02-09 04:00:00,43343.281250,43569.648438,43313.980469,43453.488281,859.031616,2022-02-09 04:59:59.999,37337804.0,30617,430.131592,...,41321.941406,5.158390,41050.941406,5.852599,40825.367188,6.437471,40640.425781,6.921833,46.468800,LONG
2022-02-09 05:00:00,2022-02-09 05:00:00,43453.488281,43749.000000,43125.000000,43584.929688,1117.572388,2022-02-09 05:59:59.999,48540336.0,-23617,554.606750,...,41347.656250,5.410883,41076.156250,6.107615,40849.789062,6.695605,40663.890625,7.183373,48.757809,LONG
2022-02-09 06:00:00,2022-02-09 06:00:00,43584.929688,43831.050781,43533.730469,43700.011719,1294.671631,2022-02-09 06:59:59.999,56554648.0,-30152,555.221130,...,41374.386719,5.620929,41102.261719,6.320212,40875.011719,6.911313,40688.082031,7.402486,50.743736,LONG


In [ ]:
numeric_features = ['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'amplitude',
                    'change_open_close', 'ema_50p', 'ema_50p_diff', 'ema_75p', 'ema_75p_diff', 'ema_100p', 'ema_100p_diff', 'ema_125p', 'ema_125p_diff', 'ema_150p', 'ema_150p_diff', 'ema_175p',
                    'ema_175p_diff', 'ema_200p', 'ema_200p_diff', 'ema_225p', 'ema_225p_diff', 'ema_250p', 'ema_250p_diff', 'rsi']
date_features = ['open_time', 'close_time']


ce = ClassificationExperiment()
setup = ce.setup(data=data,
                 train_size=0.7,
                 target=myenv.label,
                 numeric_features=numeric_features,
                 date_features=date_features,
                 create_date_columns=["hour", "day", "month"],
                 data_split_shuffle=False,
                 data_split_stratify=False,
                 fix_imbalance=True,
                 fix_imbalance_method='smoteenn',
                 feature_selection=True,
                 feature_selection_estimator='lightgbm',
                 remove_outliers=True,
                 fold_strategy='timeseries',
                 fold=3,
                 session_id=123,
                 normalize=True,
                 use_gpu=True,
                 verbose=True,
                 n_jobs=-1)

In [ ]:
# model = setup.compare_models(verbose=True)
model = setup.create_model('xgboost')
estimator = setup.pull().index[0]
print(f'Best Model Estimator: {estimator}')

In [ ]:
tuned_model = setup.tune_model(model)
tuned_model

In [ ]:
final_model = setup.finalize_model(tuned_model)
final_model

In [ ]:
predict_holds = setup.plot_model(final_model)
